### Interacting with Parse Hub 

In [2]:
import pandas as pd
import numpy as np  
from googlesearch import search
from random import randint
from time import sleep

### Load Data

In [510]:
df = pd.read_csv('Ontario_Pre.csv')

In [511]:
df.head()

,selection1_OEM,selection1_Dealer_Name_name,selection1_Dealer_Name_url,selection1_Dealer_Name_Address,selection1_Dealer_Name_City,selection1_Dealer_Name_City_url,selection1_Dealer_Name_State,selection1_Dealer_Name_State_url,selection1_Dealer_Name_ZipCode,selection1_Dealer_Name_ZipCode_url,selection1_Dealer_Name_Sales_Phone_Number,selection1_Dealer_Name_Number_DealerRaterReviews,selection1_Dealer_Name_DealerRaterRating,selection1_Dealer_Name_page
0,Chevrolet,Huron Motor Products,https://www.dealerrater.ca/dealer/Huron-Motor-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chevrolet,Finch Chevrolet Cadillac Buick GMC,https://www.dealerrater.ca/dealer/Finch-Chevro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chevrolet,Georgetown Chevrolet Buick GMC,https://www.dealerrater.ca/dealer/Georgetown-C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chevrolet,Budds' Chevrolet Cadillac Buick GMC,https://www.dealerrater.ca/dealer/Budds-Chevro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chevrolet,Wilson Niblett Motors,https://www.dealerrater.ca/dealer/Wilson-Nible...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Delete Duplicate Dealers 
Dealers created from scraping within ParseHub automatically don't have any details beside Dealer email and DealerRater URL. So locating Dealers without a zipcode and then deleting them, we delete duplicate Dealers

In [512]:
df.loc[df['selection1_Dealer_Name_ZipCode'].isnull(), 'selection1_Dealer_Name_ZipCode']='None'

In [513]:
df = df[df.selection1_Dealer_Name_ZipCode != 'None']

### Reset Index

So that we can later name the exported CSV as the State.csv

In [514]:
df.reset_index(drop=True, inplace=True)

### Rename and Reorganize Columns to main formatting

In [515]:
df = df.rename(columns={'selection1_OEM': 'Manufactuer', 'selection1_Dealer_Name_name': 'Dealership',
                        'selection1_Dealer_Name_url': 'DealerRater URL','selection1_Dealer_Name_Address': 'Address',
                       'selection1_Dealer_Name_City': 'City','selection1_Dealer_Name_State': 'State',
                       'selection1_Dealer_Name_ZipCode': 'Zip Code','selection1_Dealer_Name_Sales_Phone_Number': 'Phone Number',
                       'selection1_Dealer_Name_Number_DealerRaterReviews': 'DealerRater # of Reviews',
                        'selection1_Dealer_Name_DealerRaterRating': 'DealerRater Star Rating',
                       'selection1_Dealer_Name_page': 'Website'})

In [516]:
df['Customer'] = df.apply(lambda _: 'Customer', axis=1)
df['Google Rating'] = df.apply(lambda _: 'Google Rating', axis=1)
df['# of Google Reviews'] = df.apply(lambda _: '# of Google Reviews', axis=1)
df['Dealership Group'] = df.apply(lambda _: 'Dealership Group', axis=1)

In [517]:
df.columns

Index(['Manufactuer', 'Dealership', 'DealerRater URL', 'Address', 'City',
       'selection1_Dealer_Name_City_url', 'State',
       'selection1_Dealer_Name_State_url', 'Zip Code',
       'selection1_Dealer_Name_ZipCode_url', 'Phone Number',
       'DealerRater # of Reviews', 'DealerRater Star Rating', 'Website',
       'Customer', 'Google Rating', '# of Google Reviews', 'Dealership Group'],
      dtype='object')

In [379]:
df = df[['Manufactuer','Dealership','Address', 'City','State','Zip Code','Phone Number','Website',
      'Dealership Group','Customer', 'Google Rating','# of Google Reviews','DealerRater # of Reviews',
      'DealerRater Star Rating','DealerRater URL']]

KeyError: "['DealerRater URL', 'DealerRater # of Reviews', 'DealerRater Star Rating'] not in index"

In [519]:
cols = list(df.columns.values)

### Rename and Group all OEM's 

In [520]:
OEM_Groups = {
    'Ford': 'Ford Lincoln',
    'Lincoln': 'Ford Lincoln',
    'Dodge': 'FCA',
    'Chrysler': 'FCA',
    'Jeep': 'FCA',
    'Ram': 'FCA',
    'FIAT':'FIAT',
    'Alfa Romeo':'Alfa Romeo',
    'Buick': 'GM',
    'GMC':'GM',
    'Cadillac': 'GM',
    'Chevrolet': 'GM',
    'Toyota': 'Toyota',
    'Nissan': 'Nissan',
    'BMW': 'BMW',
    'Hyundai': 'Hyundai',
    'Kia': 'Kia',
    'Lexus': 'Lexus',
    'MINI': 'MINI',
    'Mazda': 'Mazda',
    'Mitsubishi': 'Mitsubishi',
    'Volvo': 'Volvo',
    'Honda': 'Honda',
    'Acura': 'Acura',
    'Audi': 'Audi',
    'Mercedes-Benz': 'Mercedes-Benz',
    'Subaru': 'Subaru',
    'Volkswagen': 'Volkswagen',
    'INFINITI': 'INFINITI',
    'Jaguar': 'Jaguar',
    'Porsche': 'Porsche',
    'Land Rover': 'Land Rover',
    'Maserati': 'Maserati',
    'Lamborghini': 'Lamborghini',
    'McLaren': 'McLaren',
    'Bentley': 'Bentley',
    'Aston Martin': 'Aston Martin',
    'Rolls Royce': 'Rolls Royce',
    'Recreational Vehicles': 'Recreational Vehicles',
    'Used Car Dealer': 'Used Car Dealer',
    'Ferrari': 'Ferrari',
    'smart': 'smart'
}

In [521]:
df['Manufactuer'] = df['Manufactuer'].map(OEM_Groups)

### Delete all FCA and OEM's already contained in list

Because we wanted to weed out any duplicate dealerships within the "Used Car" category, we chose to scrape all the dealerships that we already had on our list. However we already have their data, so we delete those rows

In [522]:
#df = df[df.Manufactuer != 'Ford Lincoln']
#df = df[df.Manufactuer != 'Toyota']
#df = df[df.Manufactuer != 'Nissan']
#df = df[df.Manufactuer != 'BMW']
#df = df[df.Manufactuer != 'FCA']
#df = df[df.Manufactuer != 'Hyundai']
#df = df[df.Manufactuer != 'Kia']
#df = df[df.Manufactuer != 'Lexus']
#df = df[df.Manufactuer != 'MINI']
#df = df[df.Manufactuer != 'Mazda']
#df = df[df.Manufactuer != 'Mitsubishi']

### Fill in missing Websites

In [523]:
df.loc[df['Website'].isnull(), 'Website']='None'

In [524]:
def Get_URL (platform):
    for url in search ("'" + platform + "'", stop=1):
        return(url)

In [525]:
#sleep(randint(0,3))

In [526]:
df['Website'] = np.where(df['Website'] == 'None', df['Dealership'].apply(Get_URL), df['Website'])

In [527]:
df.head()

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,DealerRater # of Reviews,DealerRater Star Rating,DealerRater URL
0,GM,Roy Nichols Chevrolet,2728 Courtice Road,Courtice,Ontario,L1E 2M7,(905) 436-2222,https://www.roynicholsmotors.com/,Dealership Group,Customer,Google Rating,# of Google Reviews,1824 Lifetime Reviews,5.0,https://www.dealerrater.ca/dealer/Roy-Nichols-...
1,GM,Jeff Smith's County Chevrolet,224 Talbot St N,Essex,Ontario,N8M 2C8,(519) 776-4222,https://www.countychevroletessex.com/,Dealership Group,Customer,Google Rating,# of Google Reviews,6 Lifetime Reviews,3.4,https://www.dealerrater.ca/dealer/Jeff-Smith-s...
2,GM,The Humberview Group,3200 Bloor Street West,Toronto,Ontario,M8X 1E1,(877) 893-0990,https://www.humberviewgroup.com/,Dealership Group,Customer,Google Rating,# of Google Reviews,1824 Lifetime Reviews,1.8,https://www.dealerrater.ca/dealer/The-Humbervi...
3,GM,Upper Canada Motor Sales Limited,12375 Country Rd 2,Morrisburg,Ontario,K0C 1X0,(613) 543-2925,https://www.uppercanadamotors.com/,Dealership Group,Customer,Google Rating,# of Google Reviews,1 Lifetime Review,1.0,https://www.dealerrater.ca/dealer/Upper-Canada...
4,GM,Bill Spencer Chevrolet Ltd,Hwy #2 West,Cobourg,Ontario,K9A 4S3,(905) 372-8773,https://www.billspencerchevrolet.com/,Dealership Group,Customer,Google Rating,# of Google Reviews,1824 Lifetime Reviews,NaN,https://www.dealerrater.ca/dealer/Bill-Spencer...


### Clear Values within Customer, Dealership Group, Google Rating and # of Google Reviews Columns

In [528]:
df[cols] = df[cols].replace({'Customer':np.nan, 'Google Rating':np.nan,
                               '# of Google Reviews':np.nan,'Dealership Group':np.nan})

### Name the CSV before we change to Abbreviations

In [529]:
State =df['State']

In [530]:
CSV_name = State[0]

### Replace States with Abbreviations

In [306]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Dist. of Columbia':'DC',
    'Newfoundland and Labrador':'NL',
    'Prince Edward Island':'PE',
    'Nova Scotia':'NS',
    'New Brunswick':'NB',
    'Quebec':'QC',
    'Ontario':'ON',
    'Manitoba':'MB',
    'Saskatchewan':'SK',
    'Alberta':'AB',
    'British Columbia':'BC',
    'Yukon':'YT',
    'Northwest Territories' :'NT',
    'Nunavut':'NU'
}

In [532]:
df['State'] = df['State'].map(us_state_abbrev)

In [533]:
df.head()

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,DealerRater # of Reviews,DealerRater Star Rating,DealerRater URL
0,GM,Roy Nichols Chevrolet,2728 Courtice Road,Courtice,ON,L1E 2M7,(905) 436-2222,https://www.roynicholsmotors.com/,NaN,NaN,NaN,NaN,1824 Lifetime Reviews,5.0,https://www.dealerrater.ca/dealer/Roy-Nichols-...
1,GM,Jeff Smith's County Chevrolet,224 Talbot St N,Essex,ON,N8M 2C8,(519) 776-4222,https://www.countychevroletessex.com/,NaN,NaN,NaN,NaN,6 Lifetime Reviews,3.4,https://www.dealerrater.ca/dealer/Jeff-Smith-s...
2,GM,The Humberview Group,3200 Bloor Street West,Toronto,ON,M8X 1E1,(877) 893-0990,https://www.humberviewgroup.com/,NaN,NaN,NaN,NaN,1824 Lifetime Reviews,1.8,https://www.dealerrater.ca/dealer/The-Humbervi...
3,GM,Upper Canada Motor Sales Limited,12375 Country Rd 2,Morrisburg,ON,K0C 1X0,(613) 543-2925,https://www.uppercanadamotors.com/,NaN,NaN,NaN,NaN,1 Lifetime Review,1.0,https://www.dealerrater.ca/dealer/Upper-Canada...
4,GM,Bill Spencer Chevrolet Ltd,Hwy #2 West,Cobourg,ON,K9A 4S3,(905) 372-8773,https://www.billspencerchevrolet.com/,NaN,NaN,NaN,NaN,1824 Lifetime Reviews,NaN,https://www.dealerrater.ca/dealer/Bill-Spencer...


### Export CSV

In [534]:
df.to_csv("" + CSV_name + ".csv",index=False, header =False)

In [ ]:
df.head()

# Getting Data Review Ready

In [3]:
from random import randint
from time import sleep
import requests
import json
from bs4 import BeautifulSoup
import re

#sleep(randint(10,100))

In [1901]:
df = pd.read_csv('Master North American OEM Dealerships - Wisconsin WI.csv')

In [1902]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address ', 'City ', 'State', 'Zip Code',
       'Phone Number', ' Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews'],
      dtype='object')

In [1903]:
#df = pd.read_csv('Test - Sheet42.csv')

In [1904]:
State =df['State']
CSV_name = State[0]
#Province = df['Province']
#CSV_name =Province[0]

In [1905]:
df["Keywords"] = df["Dealership"].map(str) + " " + df["City "] 

In [1906]:
lst = df['Dealership']
lst = [w.replace('&', 'and') for w in lst]
df['Dealership'] =lst
# & Symbols mess with the way that Google presents the dealership, so we start by replacing those symbols in the data

In [1907]:
def Gog_Rat (dealer):
    try:
        site_text = requests.get("https://google.com/search?q='" + dealer + "'").text
        soup = BeautifulSoup(site_text, 'html.parser')
        sleep(randint(1,3))
        return soup.find('div', class_='BNeawe tAd8D AP7Wnd').find('span').text
    except AttributeError:
        print ("Nope")
#This is the main code to retrieve the review data. It is extremly difficult to find a better class and span 
#that are consistently reliable. It has a random 1-3 second delay built in as a precaution. This could be reduced to 
#improve the speed

In [1908]:
df['reviews'] = df['Keywords'].apply(Gog_Rat)

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope


In [1909]:
def Get_Only_nums (dealer):
    a = re.sub("[^0-9.()]", "", str(dealer))
    return a
#With the addition of .find('span') in our Web Scrape this may not be neccesary, but it is a precaution

In [1910]:
df['Google_Review'] = df['reviews'].apply(Get_Only_nums)

In [1911]:
df['Google Rating'] = df.Google_Review.str[0:3]

In [1912]:
def Get_review (dealer):
    m= re.search('\(([^)]+)', str(dealer))
    if m is None:
        return None  
    return m.group(1)

In [1913]:
df['# of Google Reviews'] = df['Google_Review'].apply(Get_review)

In [1914]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [1915]:
df.groupby('Google Rating').size()

Google Rating
       113
.        1
1.0      1
1.5      1
2.0      2
2.6      3
2.7      1
2.8      4
2.9      2
211      1
3.0      4
3.1      4
3.2      2
3.3      6
3.4     12
3.5     13
3.6      8
3.7     17
3.8     14
3.9     18
4.0     44
4.1     50
4.2     45
4.3     77
4.4     95
4.5     92
4.6    100
4.7     59
4.8     58
4.9     11
5.0     31
dtype: int64

# Pass Through 2
### Search with Dealership and Address

In [1916]:
df.head()

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,Keywords,reviews,Google_Review
0,Acura,Zimbrick Acura,7525 Century Ave,Middleton,WI,53562,(608) 836-7776,https://www.zimbrickacura.com/,NaN,NaN,4.8,199,Zimbrick Acura Middleton,4.8 (199),4.8(199)
1,Acura,Acura of Brookfield,19180 W Bluemound Rd,Brookfield,WI,53045,(262) 785-1918,http://www.acurabrookfield.com/,NaN,NaN,4.7,290,Acura of Brookfield Brookfield,4.7 (290),4.7(290)
2,Acura,Bergstrom Acura of Appleton,2910 N Victory Ln,Appleton,WI,54913,(920) 560-2900,https://www.bergstromacura.com/,NaN,NaN,4.4,56,Bergstrom Acura of Appleton Appleton,4.4 (56),4.4(56)
3,Acura,Boyland Acura,4575 Converters Drive,Appleton,WI,54913,(920) 560-2900,https://www.dealerrater.com/dealer/Boyland-Acu...,NaN,NaN,4.4,56,Boyland Acura Appleton,4.4 (56),4.4(56)
4,Audi,Audi North Shore,4400 West Brown Deer Road,Brown Deer,WI,53223,(414) 355-4400,https://www.audinorthshore.com/,NaN,NaN,4.0,85,Audi North Shore Brown Deer,4.0 (85),4.0(85)


In [1917]:
booleanDictionary = {'nothing': 'stuff', 'Trash': 'FALSE'}

In [1918]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address ', 'City ', 'State', 'Zip Code',
       'Phone Number', ' Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Keywords', 'reviews',
       'Google_Review'],
      dtype='object')

In [1919]:
df['Google Rating'] = np.where(df['# of Google Reviews'] == 'None', df['Google Rating'].map(booleanDictionary), df['Google Rating'])
# This makes sure that Google Rating and # of Google Reviews both are unified if one of the two have 'None'


In [1920]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [1921]:
df["Keywords2"] = df["Dealership"].map(str) + " " + df["Address "]

In [1922]:
df['reviews2'] = np.where(df['Google Rating'] == 'None', 
                          df['Keywords2'].apply(Gog_Rat), df['reviews'])

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope


In [1923]:
df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews2'].str[0:4], df['Google Rating'])

In [1924]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None', df['reviews2'].apply(Get_review), 
                                     df['# of Google Reviews'])

In [1925]:
df['reviews2'] = np.where(df['Google Rating'] == 'None', df['reviews2'].apply(Get_Only_nums), df['reviews'])

In [1926]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [1927]:
df["Keywords3"] = df["Dealership"].map(str) + " " + df["Address "]+ " "+ df["City "]

In [1928]:
df.groupby('Google Rating').size()

Google Rating
 2.0      1
 2.7      1
 2.8      1
 3.3      1
 3.4      2
 3.5      2
 3.6      1
 3.7      1
 3.8      1
 3.9      3
 4.0      3
 4.1      1
 4.2      5
 4.3      1
 4.4      7
 4.5      6
 4.6      7
 4.7      4
 5.0      3
1.0       1
1.5       1
2.0       2
2.6       3
2.7       1
2.8       4
2.9       2
3.0       4
3.1       4
3.2       2
3.3       6
3.4      12
3.5      13
3.6       8
3.7      17
3.8      14
3.9      18
4.0      44
4.1      50
4.2      45
4.3      77
4.4      95
4.5      92
4.6     100
4.7      59
4.8      58
4.9      11
5.0      31
None     36
View     28
dtype: int64

# Pass Through 3
### Search with Dealership, Address, and City

In [1929]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [1930]:
df['reviews3'] = np.where(df['Google Rating'] == 'None', df['Keywords3'].apply(Gog_Rat), df['reviews2'])

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope


In [1931]:
df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews3'].str[0:4], df['Google Rating'])

In [1932]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None', df['reviews3'].apply(Get_review), 
                                     df['# of Google Reviews'])

In [1933]:
df['reviews3'] = np.where(df['Google Rating'] == 'None', df['reviews3'].apply(Get_Only_nums), df['reviews2'])

In [1934]:
df.groupby('Google Rating').size()

Google Rating
 2.0      1
 2.7      1
 2.8      1
 3.3      1
 3.4      2
 3.5      2
 3.6      1
 3.7      1
 3.8      1
 3.9      4
 4.0      4
 4.1      1
 4.2      5
 4.3      1
 4.4      8
 4.5      8
 4.6      7
 4.7      6
 4.8      1
 5.0      3
1.0       1
1.5       1
2.0       2
2.6       3
2.7       1
2.8       4
2.9       2
3.0       4
3.1       4
3.2       2
3.3       6
3.4      12
3.5      13
3.6       8
3.7      17
3.8      14
3.9      18
4.0      44
4.1      50
4.2      45
4.3      77
4.4      95
4.5      92
4.6     100
4.7      59
4.8      58
4.9      11
5.0      31
View     29
dtype: int64

# Pass Through 4 
### Only search with Dealership Name

In [1935]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [1936]:
df['reviews4'] = np.where(df['Google Rating'] == 'None', df['Dealership'].apply(Gog_Rat), df['reviews3'])

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope


In [1937]:
df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews4'].str[0:4], df['Google Rating'])

In [1938]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None',
                                     df['reviews4'].apply(Get_review), df['# of Google Reviews'])

In [1939]:
df['reviews4'] = np.where(df['Google Rating'] == 'None', df['reviews4'].apply(Get_Only_nums), df['reviews3'])

In [1940]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [1941]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [1942]:
df.groupby('Google Rating').size()

Google Rating
 2.0      1
 2.7      1
 2.8      1
 3.3      1
 3.4      2
 3.5      2
 3.6      1
 3.7      1
 3.8      2
 3.9      4
 4.0      4
 4.1      2
 4.2      6
 4.3      1
 4.4      8
 4.5      8
 4.6      8
 4.7      7
 4.8      1
 5.0      3
1.0       1
1.5       1
2.0       2
2.6       3
2.7       1
2.8       4
2.9       2
3.0       4
3.1       4
3.2       2
3.3       6
3.4      12
3.5      13
3.6       8
3.7      17
3.8      14
3.9      18
4.0      44
4.1      50
4.2      45
4.3      77
4.4      95
4.5      92
4.6     100
4.7      59
4.8      58
4.9      11
5.0      31
None     19
View     32
dtype: int64

# Clean and Export

In [1943]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address ', 'City ', 'State', 'Zip Code',
       'Phone Number', ' Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Keywords', 'reviews',
       'Google_Review', 'Keywords2', 'reviews2', 'Keywords3', 'reviews3',
       'reviews4'],
      dtype='object')

In [1944]:
del df['reviews4']
del df['Keywords']
del df['Google_Review']
del df['Keywords3']
del df['reviews']
del df['Keywords2']
del df['reviews2']
del df['reviews3']

In [1945]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None Found'
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None Found'

In [1946]:
df.to_csv("" + CSV_name + " REVIEWs.csv",index=False)

In [1947]:
df.groupby('Google Rating').size()

Google Rating
 2.0      1
 2.7      1
 2.8      1
 3.3      1
 3.4      2
 3.5      2
 3.6      1
 3.7      1
 3.8      2
 3.9      4
 4.0      4
 4.1      2
 4.2      6
 4.3      1
 4.4      8
 4.5      8
 4.6      8
 4.7      7
 4.8      1
 5.0      3
1.0       1
1.5       1
2.0       2
2.6       3
2.7       1
2.8       4
2.9       2
3.0       4
3.1       4
3.2       2
3.3       6
3.4      12
3.5      13
3.6       8
3.7      17
3.8      14
3.9      18
4.0      44
4.1      50
4.2      45
4.3      77
4.4      95
4.5      92
4.6     100
4.7      59
4.8      58
4.9      11
5.0      31
None     19
View     32
dtype: int64

Possibly needs one with Dealership + State?

In [ ]:
df.head(75)

# Duplicate 

In [608]:
df = pd.read_csv('Master North American OEM Dealerships - Quebec QC.csv')

In [609]:
State =df['State']
CSV_name = State[0]

KeyError: 'State'

In [ ]:
df["Keywords"] = df["Dealership"].map(str) + " " + df["City "] 

In [ ]:
lst = df['Dealership']
lst = [w.replace('&', 'and') for w in lst]
df['Dealership'] =lst

In [ ]:
def Gog_Rat (dealer):
    try:
        site_text = requests.get("https://google.com/search?q='" + dealer + "'").text
        soup = BeautifulSoup(site_text, 'html.parser')
        sleep(randint(1,3))
        return soup.find('div', class_='BNeawe tAd8D AP7Wnd').find('span').text
    except AttributeError:
        print ("Nope")

In [ ]:
df['reviews'] = df['Keywords'].apply(Gog_Rat)

In [ ]:
def Get_Only_nums (dealer):
    a = re.sub("[^0-9.()]", "", str(dealer))
    return a

In [ ]:
df['Google_Review'] = df['reviews'].apply(Get_Only_nums)


In [ ]:
df['Google Rating'] = df.Google_Review.str[0:3]

In [ ]:
def Get_review (dealer):
    m= re.search('\(([^)]+)', str(dealer))
    if m is None:
        return None  
    return m.group(1)

In [ ]:
df['# of Google Reviews'] = df['Google_Review'].apply(Get_review)
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [ ]:
booleanDictionary = {'nothing': 'stuff', 'Trash': 'FALSE'}

In [ ]:
df['Google Rating'] = np.where(df['# of Google Reviews'] == 'None', df['Google Rating'].map(booleanDictionary), df['Google Rating'])


In [ ]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

df["Keywords2"] = df["Dealership"].map(str) + " " + df["Address "]

In [ ]:
df['reviews2'] = np.where(df['Google Rating'] == 'None', df['Keywords2'].apply(Gog_Rat), df['reviews'])

In [ ]:
df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews2'].str[0:4], df['Google Rating'])

In [ ]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None', df['reviews2'].apply(Get_review), 
                                     df['# of Google Reviews'])

In [ ]:
df['reviews2'] = np.where(df['Google Rating'] == 'None', df['reviews2'].apply(Get_Only_nums), df['reviews'])
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [ ]:
df["Keywords3"] = df["Dealership"].map(str) + " " + df["Address "]+ " "+ df["City "]

In [ ]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [ ]:
df['reviews3'] = np.where(df['Google Rating'] == 'None', df['Keywords3'].apply(Gog_Rat), df['reviews2'])

In [ ]:

df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews3'].str[0:4], df['Google Rating'])

In [ ]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None', df['reviews3'].apply(Get_review), df['# of Google Reviews'])

In [ ]:
df['reviews3'] = np.where(df['Google Rating'] == 'None', df['reviews3'].apply(Get_Only_nums), df['reviews2'])

In [ ]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [ ]:
df['reviews4'] = np.where(df['Google Rating'] == 'None', df['Dealership'].apply(Gog_Rat), df['reviews3'])

In [ ]:
df['Google Rating'] = np.where(df['Google Rating'] == 'None', df['reviews4'].str[0:4], df['Google Rating'])

In [ ]:
df['# of Google Reviews'] = np.where(df['# of Google Reviews']=='None',
                                     df['reviews4'].apply(Get_review), df['# of Google Reviews'])

In [ ]:
df['reviews4'] = np.where(df['Google Rating'] == 'None', df['reviews4'].apply(Get_Only_nums), df['reviews3'])

In [ ]:
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [ ]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None'

In [ ]:
df.groupby('Google Rating').size()

In [ ]:
del df['reviews4']
del df['Keywords']
del df['Google_Review']
del df['Keywords3']
del df['reviews']
del df['Keywords2']
del df['reviews2']
del df['reviews3']

In [ ]:
df.loc[df['# of Google Reviews'].isnull(), '# of Google Reviews']='None Found'
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None Found'

In [ ]:
df.to_csv("" + CSV_name + " REVIEWs.csv",index=False)

### Possibly using Selenium to find if a text/code exists on a webpage

https://stackoverflow.com/questions/11454798/how-can-i-check-if-some-text-exist-or-not-in-the-page-using-selenium/11464797

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException


In [9]:
driver = webdriver.Chrome("/Users/zacharychild/Desktop/chromedriver")

### Possible issue with faulty websites (DealerRater etc...)

In [1109]:
driver.get("https://www.google.com/search?ei=IM0nXc2tPKHN0PEP4teAwAQ&q=parker+ford+Coeur+D%27Alene&oq=parker+ford+Coeur+D%27Alene&gs_l=psy-ab.3..0l6j0i67j0l3.6085.6255..6381...0.0..0.95.95.1......0....1j2..gws-wiz.......0i71.L4L69vQElR4")

In [37]:
df = pd.read_csv('First100 CT.csv')

In [38]:
df.head()

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,Geographic Coordinates
0,Acura,Greenwich Acura,343 W Putnam Ave,Greenwich,CT,6830,(203) 625-8200,https://www.greenwichacura.com/,NaN,NaN,4.6,31,"(41.0233562660145, -73.6377293025713)"
1,Acura,Acura of Berlin,245 Webster Square Road,Berlin,CT,6037,(877) 621-6916,https://www.acuraofberlin.com/,NaN,NaN,4.5,298,"(41.6299, -72.744464)"
2,Acura,Acura by Executive,527 Washington Ave,North Haven,CT,6473,(888) 993-7059,https://www.acurabyexecutive.com/,NaN,NaN,4.5,134,"(41.4203123, -72.8399352)"
3,Acura,Acura of Avon,75 Albany Tpke.,Canton,CT,6019,(860) 693-6981,https://www.acuraofavon.com/,NaN,NaN,4.5,120,"(41.8240336, -72.880002953792)"
4,Acura,Acura of Milford,1503 Boston Post Rd,Milford,CT,6460,(203) 877-4333,https://www.acuraofmilford.com/,NaN,NaN,3.8,165,"(41.2431623, -73.0295306880799)"


In [32]:
def convert(url):
    if url.startswith('http://www.'):
        return 'http://' + url[len('http://www.'):]
    if url.startswith('www.'):
        return 'http://' + url[len('www.'):]
    if not url.startswith('http://'):
        return 'http://' + url
    return url

In [33]:
ChromeOptions options = new ChromeOptions();
               options.addArguments("no-sandbox");

SyntaxError: invalid syntax (<ipython-input-33-6c0fc07ba09f>, line 1)

In [42]:
df.columns


Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates'],
      dtype='object')

In [49]:

#options.add_argument('no-sandbox')

def Check_Chat (Website):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(executable_path="/Users/zacharychild/Desktop/chromedriver 3", options=options)
    driver.get("" + Website + "")
    sleep(randint(2,3))
    m= driver.page_source
    try:
        if ("cars2go" in driver.page_source):
            return "cars2go"   
        if ("gubagoo" in driver.page_source):
            return "Gubagoo"
        if ("ActivEngageChatButton" in driver.page_source):
            return "ActivEngage"
        if ("carnow-plugin" in driver.page_source):
            return "CarNow"
        if ("CarNowPlugin" in driver.page_source):
            return "CarNow"
        if ("carcodesms" in driver.page_source):
            return "Car Code--Edmunds"
        if ("carchat24" in driver.page_source):
            return "CarChat24"
        if ("di-conversations" in driver.page_source):
            return "Dealer Inspire"
        if ("engagetosell" in driver.page_source):
            return "Engage To Sell"
        if ("dealereprocesschat" in driver.page_source):
            return "Dealer Eprocess"
        if ("contactatonce" in driver.page_source):
            return "Contact At Once"
        if ("captcha" in driver.page_source):
            return "Captcha Blocked"
    except InvalidArgumentException:
        print ("URL ERROR")
        pass
    except WebDriverException:
        print ("URL ERROR2")
        pass
    driver.quit() 
    #driver.close() 
    
   

In [40]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates'],
      dtype='object')

In [43]:
df['Chat'] = df['Website'].apply(Check_Chat)

In [51]:
for i, row in enumerate(df):
    try:
        df['Website'].apply(Check_Chat)
    except Exception as e: 
        print('Error at index {}: {!r}'.format(i, row))
        print(e)

Error at index 2: 'Address'
Message: unknown error: unable to discover open pages

Error at index 3: 'City'
Message: unknown error: unable to discover open pages

Error at index 4: 'State'
Message: unknown error: unable to discover open pages

Error at index 5: 'Zip Code'
Message: unknown error: unable to discover open pages

Error at index 9: 'Customer'
Message: unknown error: unable to discover open pages

Error at index 10: 'Google Rating'
Message: unknown error: unable to discover open pages

Error at index 11: '# of Google Reviews'
Message: unknown error: unable to discover open pages

Error at index 12: 'Geographic Coordinates'
Message: unknown error: unable to discover open pages

Error at index 13: 'Chat'
Message: unknown error: unable to discover open pages



In [ ]:
def Check_Chat (Website):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(executable_path="/Users/zacharychild/Desktop/chromedriver 3", options=options)
    driver.get("" + Website + "")
    sleep(randint(2,3))
    m= driver.page_source
    try:
        if ("cars2go" in driver.page_source):
            return "cars2go"   
        if ("gubagoo" in driver.page_source):
            return "Gubagoo"
        if ("ActivEngageChatButton" in driver.page_source):
            return "ActivEngage"
        if ("carnow-plugin" in driver.page_source):
            return "CarNow"
        if ("CarNowPlugin" in driver.page_source):
            return "CarNow"
        if ("carcodesms" in driver.page_source):
            return "Car Code--Edmunds"
        if ("carchat24" in driver.page_source):
            return "CarChat24"
        if ("di-conversations" in driver.page_source):
            return "Dealer Inspire"
        if ("engagetosell" in driver.page_source):
            return "Engage To Sell"
        if ("dealereprocesschat" in driver.page_source):
            return "Dealer Eprocess"
        if ("contactatonce" in driver.page_source):
            return "Contact At Once"
        if ("captcha" in driver.page_source):
            return "Captcha Blocked"
    except InvalidArgumentException:
        print ("URL ERROR")
        pass
    except WebDriverException:
        print ("URL ERROR2")
        pass
    driver.quit()

Check on first 100

In [53]:
df.head(50)

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,Geographic Coordinates,Chat
0,Acura,Greenwich Acura,343 W Putnam Ave,Greenwich,CT,6830,(203) 625-8200,https://www.greenwichacura.com/,NaN,NaN,4.6,31,"(41.0233562660145, -73.6377293025713)",None
1,Acura,Acura of Berlin,245 Webster Square Road,Berlin,CT,6037,(877) 621-6916,https://www.acuraofberlin.com/,NaN,NaN,4.5,298,"(41.6299, -72.744464)",Gubagoo
2,Acura,Acura by Executive,527 Washington Ave,North Haven,CT,6473,(888) 993-7059,https://www.acurabyexecutive.com/,NaN,NaN,4.5,134,"(41.4203123, -72.8399352)",Gubagoo
3,Acura,Acura of Avon,75 Albany Tpke.,Canton,CT,6019,(860) 693-6981,https://www.acuraofavon.com/,NaN,NaN,4.5,120,"(41.8240336, -72.880002953792)",None
4,Acura,Acura of Milford,1503 Boston Post Rd,Milford,CT,6460,(203) 877-4333,https://www.acuraofmilford.com/,NaN,NaN,3.8,165,"(41.2431623, -73.0295306880799)",None
5,Acura,Devan Acura of Norwalk,625 West Ave,Norwalk,CT,6850,(888) 282-4525,https://www.devanacura.com/,NaN,NaN,4.5,116,NaN,None
6,Acura,Antonino Acura,340 Gold Star Hwy,Groton,CT,6340,(860) 448-1299,https://www.antoninoacura.com/,NaN,NaN,4.3,121,"(41.370364913683, -72.0596413054629)",Car Code--Edmunds
7,Acura,Schaller Acura,345 Center St,Manchester,CT,6040,(866) 780-8169,https://www.schalleracura.com/,NaN,NaN,4.7,171,"(41.772521, -72.535054)",Gubagoo
8,Alfa Romeo,Alfa Romeo of Danbury,52 Newtown Road,Danbury,CT,6810,(888) 814-2986,https://www.danburyalfaromeo.com/,NaN,NaN,4.6,54,"(41.403798, -73.423425)",None
9,Audi,Audi Danbury,25 Sugar Hollow Rd,Danbury,CT,6810,(203) 744-5202,https://www.audidanbury.com/,NaN,NaN,4.8,480,"(41.3759855, -73.473856)",Car Code--Edmunds


In [52]:
df.to_csv("IDK WHAT THIS IS Chat.csv",index=False)

# TRY THIS TOMMOROW 

In [55]:
for i, row in enumerate(df['Website']):
    try:
        df['Website'].apply(Check_Chat)
    except Exception as e: 
        print('Error at index {}: {!r}'.format(i, row))
        print(e)

Error at index 3: 'https://www.acuraofavon.com/'
Message: chrome not reachable

Error at index 5: 'https://www.devanacura.com/'
Message: unknown error: unable to discover open pages

Error at index 6: 'https://www.antoninoacura.com/'
Message: unknown error: unable to discover open pages

Error at index 7: 'https://www.schalleracura.com/'
Message: unknown error: unable to discover open pages

Error at index 8: 'https://www.danburyalfaromeo.com/'
Message: unknown error: unable to discover open pages

Error at index 9: 'https://www.audidanbury.com/'
Message: timeout
  (Session info: headless chrome=75.0.3770.142)

Error at index 11: 'https://www.hoffmanaudiofnewlondon.com/'
Message: unknown error: unable to discover open pages

Error at index 12: 'https://www.hoffmanaudiofhartford.com/'
Message: unknown error: unable to discover open pages

Error at index 13: 'https://www.valentiaudi.com/'
Message: unknown error: unable to discover open pages

Error at index 14: 'https://www.audifairfield

KeyboardInterrupt: 

In [ ]:
"driver.manage().timeouts().implicitlyWait()""

In [147]:
driver = webdriver.Chrome("/Users/zacharychild/Desktop/chromedriver", options=options)
driver.get("https://www.pricedrightauto.org/newandusedcars.aspx/")
sleep(randint(1,3))
m= driver.page_source

In [148]:
m

'<html xmlns="http://www.w3.org/1999/xhtml"><head><title>\n\tUsed Cars Marne MI | Used Cars &amp; Trucks MI | Priced Right Auto\n</title><link rel="Stylesheet" type="text/css" href="https://www.pricedrightauto.org/CSS/font-awesome.css"><link rel="Stylesheet" type="text/css" href="https://www.pricedrightauto.org/CSS/animate.css"><link rel="Stylesheet" type="text/css" href="https://www.pricedrightauto.org/CSS/bootstrap.min.css"><link rel="stylesheet" type="text/css" href="https://www.pricedrightauto.org/Modules/Address and Phone/module.css"><link rel="stylesheet" type="text/css" href="https://www.pricedrightauto.org/Modules/Social Media/module.css"><link rel="stylesheet" type="text/css" href="https://www.pricedrightauto.org/Modules/Footer Nav/module.css"><link rel="stylesheet" type="text/css" href="https://www.pricedrightauto.org/Modules/Copyright/module.css"><link rel="stylesheet" type="text/css" href="https://www.pricedrightauto.org/Modules/SEO/module.css"><link rel="stylesheet" type="

In [264]:
driver.close() 

In [265]:
driver.getPageSource().contains("Text which you looking for");


AttributeError: 'WebDriver' object has no attribute 'getPageSource'

In [1992]:
"https://www.allannotttoyota.com/"

'https://www.allannotttoyota.com/'

In [7]:
import requests
from bs4 import BeautifulSoup

headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

reddit1Link = requests.get("https://www.billkayford.com/", headers=headers)
reddit1Content =BeautifulSoup(reddit1Link.content,"lxml")
print(reddit1Content)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta id="captcha-bypass" name="captcha-bypass"/>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" media="screen,projection" rel="stylesheet" type="text/css"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" type="text/css" media="screen,projection" /><![endif]-->
<style typ

In [5]:
site_text = requests.get("https://novadentist.com/").text
soup = BeautifulSoup(site_text, 'html.parser')

In [6]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUAUVNXDRACU1BUBAIHXw=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(18),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,n,e){r(e.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,n){return t}).join(", ")))},{}],2:[function(t,n,e){function 

In [266]:
dealer ="Newark Toyotaworld-Scion"
edmunds-button

In [72]:
try:
    site_text = requests.get("https://google.com/search?q='" + dealer + "'").text
    soup = BeautifulSoup(site_text, 'html.parser')
    sleep(randint(1,2))
    print(soup.find('div', class_='BNeawe tAd8D AP7Wnd').find('span').text)
except AttributeError:
        print ("Nope")

Nope


" type="text/javascript" src="//cdn3.dealereprocesschat.com/assets/dealereprocess/davidstanleydodge.com/chatloader.min.js">"

DealerEProcess

# Start of Improving Address Code

In [4]:
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [5]:
nom=Nominatim()

/Users/zacharychild/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [215]:
df = pd.read_csv('Master North American OEM Dealerships - Alabama AL.csv')

In [216]:
df = df.rename(columns={'Manufactuer': 'Manufactuer', 'Dealership': 'Dealership','Address ': 'Address',
                       'City ': 'City','State': 'State',
                       'Zip Code': 'Zip Code','Phone Number': 'Phone Number',
                       ' Website': 'Website',
                       'Dealership Group': 'Dealership Group',
                       'Customer': 'Customer',
                       'Google Rating': 'Google Rating',
                       '# of Google Reviews': '# of Google Reviews',
                       'Geographic Coordinates': 'Geographic Coordinates'
                       })

In [217]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates'],
      dtype='object')

In [218]:
State =df['State']
CSV_name = State[0]

In [219]:
original_Address = df['Address']
original_City = df['City']
original_Zip_Code = df['Zip Code']

In [220]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates'],
      dtype='object')

In [221]:
def Geo_Cords (Address):
    try:
        n = nom.geocode("'" + Address + "'", timeout=10)
        time.sleep(0.5)
        return (n.latitude, n.longitude)
        pass
    except AttributeError:
        print ("Nope")
        pass
    
    except GeocoderTimedOut:
        print("Error: geocode failed on input %s with message %s")
        pass
    
    except TypeError:
        print("Error: geocode failed")
        pass
    

In [222]:
df["GeoSearchTerm"] = df["Address"].map(str) + ", " + df["City"]+ " "+ df["State"]

In [223]:
df['Geo Cords'] = df['GeoSearchTerm'].apply(Geo_Cords)

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope


In [224]:
def Update_address (Dealership):
    try:
        site_text = requests.get("https://google.com/search?q='" + Dealership + "'").text
        soup = BeautifulSoup(site_text, 'html.parser')
        sleep(randint(1,2))
        return(soup.find('div', class_='BNeawe s3v9rd AP7Wnd').text)
    except AttributeError:
        print ("Nope")

In [225]:
lst = df['Dealership']
lst = [w.replace('&', 'and') for w in lst]
df['Dealership'] =lst

In [226]:
df.loc[df['Geo Cords'].isnull(), 'Geo Cords']='None'

In [227]:
df["Keywords45"] = df["Dealership"].map(str) + " " +  df["City"]+ " " +  df["State"]

In [228]:
df['Keywords45'] = np.where(df['Geo Cords']=='None',
                                     df['Keywords45'].apply(Update_address), df['Keywords45'])

In [229]:
df.head(25)

,Manufactuer,Dealership,Address,City,State,Zip Code,Phone Number,Website,Dealership Group,Customer,Google Rating,# of Google Reviews,Geographic Coordinates,GeoSearchTerm,Geo Cords,Keywords45
0,Acura,Joe Bullard Acura,1151 East Interstate 65 Service Rd S,Mobile,AL,36606,(251) 263-8847,https://www.joebullardacura.com/,NaN,NaN,4.6,127,NaN,"1151 East Interstate 65 Service Rd S, Mobile AL",None,Address: 1151 East Interstate 65 Service Rd S ...
1,Acura,Jerry Damson Acura,2402 Leeman Ferry Rd SW,Huntsville,AL,35805,(888) 788-6757,https://www.jerrydamsonacura.com/,NaN,NaN,4.5,221,"(34.712892693667, -86.5915541160048)","2402 Leeman Ferry Rd SW, Huntsville AL","(34.712892693667, -86.5915541160048)",Jerry Damson Acura Huntsville AL
2,Acura,King Acura,1687 Montgomery Hwy,Hoover,AL,35216,(888) 468-0553,https://www.kingacura.com/,NaN,NaN,4.8,386,"(33.393571, -86.806251)","1687 Montgomery Hwy, Hoover AL","(33.393571, -86.806251)",King Acura Hoover AL
3,Acura,McConnell Acura,2860 Eastern Blvd,Montgomery,AL,36116,(334) 271-5551,https://www.mcconnellacura.com/,NaN,NaN,4.6,65,"(32.3424147452054, -86.2248324307946)","2860 Eastern Blvd, Montgomery AL","(32.3424147452054, -86.2248324307946)",McConnell Acura Montgomery AL
4,Alfa Romeo,Alfa Romeo of Huntsville,6523 University Dr NW Suite A,Huntsville,AL,35806,(256) 684-8342,https://www.alfaromeousaofhuntsville.com/,NaN,NaN,4.6,10,NaN,"6523 University Dr NW Suite A, Huntsville AL",None,"Address: 6523 University Dr NW Suite A, Huntsv..."
5,Audi,Audi Birmingham,1314 Grants Mill Way,Irondale,AL,35210,(855) 627-3378,https://www.audiofbirmingham.com/,NaN,NaN,4.5,252,NaN,"1314 Grants Mill Way, Irondale AL",None,"Address: 1314 Grants Mill Way, Irondale, AL 35210"
6,Audi,Audi Mobile,1525 East Interstate 65 Service Rd S Suite C,Mobile,AL,36606,(855) 768-8306,https://www.audimobileal.com/,NaN,NaN,4.1,119,NaN,"1525 East Interstate 65 Service Rd S Suite C, ...",None,Address: 1525 East Interstate 65 Service Rd S ...
7,Bentley,Bentley Pontiac Cadillac GMC Saab,2120 Drake Ave SW,Huntsville,AL,35805,(256) 275-4366,https://www.bentleygmc.com/,NaN,NaN,4.6,520,"(34.7053313210359, -86.5932563359401)","2120 Drake Ave SW, Huntsville AL","(34.7053313210359, -86.5932563359401)",Bentley Pontiac Cadillac GMC Saab Huntsville AL
8,BMW,BMW of Birmingham,1000 Tom Williams Way,Irondale,AL,35210,(205) 252-9512,https://www.tomwilliamsBMW.com,Sonic Automotive Inc.,NaN,4.4,823,NaN,"1000 Tom Williams Way, Irondale AL",None,"Address: 1000 Tom Williams Way, Irondale, AL 3..."
9,BMW,BMW of Montgomery,731 Eastern Blvd,Montgomery,AL,36117,(800) 996-9705,https://www.bmwofmontgomery.com,Sonic Automotive Inc.,NaN,4.7,613,"(32.370355979427, -86.2095771115284)","731 Eastern Blvd, Montgomery AL","(32.370355979427, -86.2095771115284)",BMW of Montgomery Montgomery AL


In [230]:
df['Keywords45'] = np.where(df['Geo Cords'] == 'None', df['Keywords45'].str[9:], df['Keywords45'])

In [231]:
new = df['Keywords45'].str.split(",",n=3, expand =True)   

In [232]:
def check_length (inputs):
    if len(inputs)>= 60: 
        return "1"

In [233]:
df['Banana'] = df['Keywords45'].apply(check_length)

In [234]:
df.loc[df['Banana'].isnull(), 'Banana']='None'

In [235]:
Updated_address = np.where(df['Banana']!="1", new[0], df['Address'])
Updated_City =np.where(df['Banana']!="1", new[1], df['City'])
Updated_Zipcode =np.where(df['Banana']!="1", new[2].str[3:], df['Zip Code'])
#Updated_Zipcode =new[2].str[3:]

In [236]:
df['Address'] = np.where(df['Geo Cords'] == 'None', Updated_address, df['Address'])
df['City'] = np.where(df['Geo Cords'] == 'None', Updated_City, df['City'])
df['Zip Code'] = np.where(df['Geo Cords'] == 'None', Updated_Zipcode, df['Zip Code'])

In [237]:
df["GeoSearchTerm"] = df["Address"].map(str) + ", " + df["City"]+ " "+ df["State"]

In [238]:
df['Geo Cords2'] = np.where(df['Geo Cords']=='None', df['GeoSearchTerm'].apply(Geo_Cords), df['Geo Cords'])

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Error: geocode failed
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Error: geocode failed
Nope
Nope
Nope
Error: geocode failed
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Error: geocode failed
Error: geocode failed
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Error: geocode failed
Error: geocode failed


In [239]:
#df['Geo Cords3'] = np.where(df['Geo Cords2']=='None', df['GeoSearchTerm'].apply(Geo_Cords), df['Geo Cords2'])
df.columns      

Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates',
       'GeoSearchTerm', 'Geo Cords', 'Keywords45', 'Banana', 'Geo Cords2'],
      dtype='object')

In [240]:
del df['GeoSearchTerm']
del df['Geo Cords']
del df['Keywords45']
del df['Banana']

In [241]:
df = df.rename(columns={'Geo Cords2': 'Geographic Coordinates'
                        })

In [242]:
#df = pd.read_csv('Master North American OEM Dealerships - Hawaii HI.csv')

In [243]:
booleanDictionary = {'View': 'None', 'None Found': 'None',
                    '1.0': '1.0',
                    '1.1': '1.1',
                    '1.2': '1.2',
                    '1.3': '1.3',
                    '1.4': '1.4',
                    '1.5': '1.5',
                    '1.6': '1.6',
                    '1.7': '1.7',
                    '1.8': '1.8',
                    '1.9': '1.9',
                    '2.0': '2.0',
                    '2.1': '2.1',
                    '2.2': '2.2',
                    '2.3': '2.3',
                    '2.4': '2.4',
                    '2.5': '2.5',
                    '2.6': '2.6',
                    '2.7': '2.7',
                    '2.8': '2.8',
                    '2.9': '2.9',
                    '3.0': '3.0',
                    '3.1': '3.1',
                    '3.2': '3.2',
                    '3.3': '3.3',
                    '3.4': '3.4',
                    '3.5': '3.5',
                    '3.6': '3.6',
                    '3.7': '3.7',
                    '3.8': '3.8',
                    '3.9': '3.9',
                    '4.0': '4.0',
                    '4.1': '4.1',
                    '4.2': '4.2',
                    '4.3': '4.3',
                    '4.4': '4.4',
                    '4.5': '4.5',
                    '4.6': '4.6',
                    '4.7': '4.7',
                    '4.8': '4.8',
                    '4.9': '4.9',
                    '5.0': '5.0'}


In [244]:
df['Google Rating'] = df['Google Rating'].map(booleanDictionary)
df.loc[df['Google Rating'].isnull(), 'Google Rating']='None'

In [245]:
#df = pd.read_csv('ThiS.csv')

In [246]:
df.columns

Index(['Manufactuer', 'Dealership', 'Address', 'City', 'State', 'Zip Code',
       'Phone Number', 'Website', 'Dealership Group', 'Customer',
       'Google Rating', '# of Google Reviews', 'Geographic Coordinates',
       'Geographic Coordinates'],
      dtype='object')

In [247]:
def Get_Only_nums (dealer):
    a = re.sub("[^0-9.()]", "", str(dealer))
    return a

In [248]:
df['Zip Code'] = df['Zip Code'].apply(Get_Only_nums)

In [249]:
#df.loc[df['Zip Code'].isnull(), 'Zip Code']='None'

In [250]:
df['fartnugget'] = np.where(df['Zip Code'] == '', "1", "0")

In [251]:
df['Address'] = np.where(df['fartnugget'] == "1", original_Address, df['Address'])
df['City'] = np.where(df['fartnugget'] == "1", original_City, df['City'])
df['Zip Code'] = np.where(df['fartnugget'] == "1", original_Zip_Code, df['Zip Code'])
del df['fartnugget']

In [252]:
df['fartnugget2'] = np.where(df['Zip Code'] == '.', "1", "0")

In [253]:
df['Address'] = np.where(df['fartnugget2'] == "1", original_Address, df['Address'])
df['City'] = np.where(df['fartnugget2'] == "1", original_City, df['City'])
df['Zip Code'] = np.where(df['fartnugget2'] == "1", original_Zip_Code, df['Zip Code'])
del df['fartnugget2']

In [254]:
df.to_csv("" + CSV_name + " Improved Address.csv",index=False)

In [255]:
df['Zip Code'][6]

'36606'

# End of Improving Address Code

In [131]:
def check_length (inputs):
    if len(inputs)>= 60: 
        pass

In [132]:
Dealership = 'Salmon River Motors Salmon ID'

In [1]:
site_text = requests.get("https://www.fremontfordcody.com/").text
soup = BeautifulSoup(site_text, 'html.parser')
        

NameError: name 'requests' is not defined

In [1952]:
soup

<!DOCTYPE html>
 <html lang="en"> <head> <meta charset="utf-8"/> <title>Ford Dealer in Cody, WY | Used Cars Cody | Fremont Ford Cody </title> <!-- Meta Tags --> <meta content="Fremont Ford Cody is your source for new Fords and used cars in Cody, WY. Browse our full inventory online and then come down for a test drive." name="description"/> <meta content="telephone=no" name="format-detection"/> <meta content="index,follow,noydir,noodp" name="robots"/> <meta content="44.533470153808594,-109.02782440185547" name="ICBM"/> <meta content="44.533470153808594,-109.02782440185547" name="geo.position"/> <meta content="Cody" name="geo.placename"/> <meta content="US-WY" name="geo.region"/> <meta content="width=device-width, initial-scale=1.0" name="viewport"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="Fremont Ford Cody is your source for new Fords and used cars in Cody, WY. Browse our full inventory online and then come down for a test drive." property="twitter:card"/> 

In [1152]:
banana = "1090 ford way blackfoot id 83221"

In [1381]:
n = nom.geocode("30 Gun Club Rd, Sagle, ID 83860")

In [1382]:
n.latitude

AttributeError: 'NoneType' object has no attribute 'latitude'

In [1123]:
df = pd.read_csv('Test - Sheet52.csv')

In [1124]:
df.columns

Index(['Example'], dtype='object')

In [1129]:
df['New'] = df['Example'].str[9:]

In [1130]:
df['New'][1
         ]

'6021 E Black Horse Pike, Egg Harbor Township, NJ 08234'

In [1138]:
my_string = "6021 E Black Horse Pike, Egg Harbor Township, NJ 08234"
result = [x.strip() for x in my_string.split(',')]
print(result[1])

Egg Harbor Township


In [1950]:
site_text

<Response [200]>

In [322]:
df = pd.read_csv('MyMap PA.csv')

In [323]:
df["Complete Address"] = df["Address"].map(str) + ", " + df["City"]+ " "+ df["State"]

In [324]:
df.to_csv("Map PA1.csv",index=False)